## Asynco with Yield

In [2]:
import os
import time 

def clock():
    time0 = round(time.time())
    while True:
        if round(time.time() - time0) % 5 == 0:
            yield '5 sec'
        else:
            yield 0
            
def query():
    for path in os.walk('/Users/'):
        yield path[0]
    
def main():
    query_gen = query()
    clock_gen = clock()
    
    while True:
        q = next(query_gen)
        c = next(clock_gen)
        print(q)
        if c: print(c)
        time.sleep(1)
        
   
main()
        
    
    

/Users/
5 sec
/Users/Shared
/Users/Shared/Previously Relocated Items 1
/Users/Shared/Previously Relocated Items 1/Configuration
/Users/Shared/Previously Relocated Items 1/Configuration/private
/Users/Shared/Previously Relocated Items 1/Configuration/private/etc
5 sec
/Users/Shared/Previously Relocated Items 1/Configuration/private/etc/ssh.system_default
/Users/Shared/Previously Relocated Items 1/Configuration/private/etc/ssh.system_default/crypto
/Users/Shared/Previously Relocated Items 1/Configuration/private/etc/ssh.system_default/ssh_config.d
/Users/Shared/Previously Relocated Items 1/Configuration/private/etc/ssh.system_default/sshd_config.d
/Users/Shared/SC Info
5 sec
/Users/Shared/Relocated Items
/Users/Shared/Relocated Items/Configuration
/Users/Shared/Relocated Items/Configuration/private
/Users/Shared/Relocated Items/Configuration/private/etc
/Users/Shared/Relocated Items/Configuration/private/etc/ssh.system_default
5 sec
/Users/Shared/Relocated Items/Configuration/private/etc

KeyboardInterrupt: 

## Asyncio 

In [ ]:
import asyncio

async def print1():
    print(1)
    
async def print2():
    await asyncio.sleep(3)
    print(2)
    
async def print3():
    print(3)
    
async def main():
    async with asyncio.TaskGroup() as tg:
        tg.create_task(print1())
        tg.create_task(print2())
        tg.create_task(print3())
    
# async def main():
#     task1 = asyncio.create_task(print1())
#     task2 = asyncio.create_task(print2())
#     task3 = asyncio.create_task(print3())
    
#     # await task1
#     # await task2
#     # await task3
    
#     # await asyncio.gather(task1, task2, task3)

#     tasks = [task1, task2, task3]
#     await asyncio.gather(*tasks)
    
await main()

1
3
2


## Asyncio Example 2

In [ ]:
async def do_something(sec):
    await asyncio.sleep(sec)
    print(f"result = {sec}")
    
async def print1(sec):
    await asyncio.sleep(sec)
    print(sec)
    await do_something(sec)
    
async def main():
    async with asyncio.TaskGroup() as tg:
        for i in range(1, 16):
            tg.create_task(print1(i))
            
await main()

1
2
result = 1
3
4


CancelledError: 

## Asyncio counter

In [3]:
from functools import wraps

async def count1(n):
    print(f'start {n}')
    await asyncio.sleep(n)
    print(f'finish {n}')
    
async def count2(n):
    print(f'start {n}')
    await asyncio.sleep(n)
    print(f'finish {n}')
    
async def count3(n):
    print(f'start {n}')
    await asyncio.sleep(n)
    print(f'finish {n}')
    

def timer(func):
    @wraps(func)
    async def wrapper(*args, **kwrags):
        start = time.time()
        result = await func(*args, **kwrags)
        end = time.time()
        print(f'Result for {func.__name__} = {end-start:.2f}')
        return result
    return wrapper


@timer
async def main_order():
    await count1(2)
    await count2(3)
    await count3(1)
    
    
@timer    
async def main():
    task1 = asyncio.create_task(count1(2))
    task2 = asyncio.create_task(count2(3))
    task3 = asyncio.create_task(count3(1))
    await asyncio.gather(task1, task2, task3)
    
await main()
await main_order()
# asyncio.run(main())
# asyncio.ran(main_order())
    


NameError: name 'asyncio' is not defined

## Task Cancel

In [ ]:
async def long_task(n):
    
    try:
        await asyncio.sleep(n)
    except asyncio.CancelledError:
        print("Cancel verified - catch exception")
    
    print("Завершено")
    
async def main():
    task1 = asyncio.create_task(long_task(5))
    
    await asyncio.sleep(1)
    print("Time is up. Canceling...")
    task1.cancel()
    
    await asyncio.gather(task1)
    
await main()

Time is up. Canceling...
Cancel verified - catch exception
Завершено


## Asyncio wait_for

In [ ]:
async def delayed_func(n):
    await asyncio.sleep(n)
    print("Завершено")
    
    
async def main():
    try:
        await asyncio.wait_for(delayed_func(5), 2)
    except asyncio.TimeoutError:
        print("Не уложился")
        
await main()

Не уложился


## Asyncio queue

In [5]:
import asyncio

queue = asyncio.Queue()

async def producer():
    for i in range(5):
        await asyncio.sleep(1)
        await queue.put(i)
        print(f"Положили {i} в очередь")
        
async def consumer():
    while True:
        item = await queue.get()
        print(f"Обрабатываем {item}")
        asyncio.sleep(2)
        queue.task_done()
        
async def main():
    prod = asyncio.create_task(producer())
    cons = asyncio.create_task(consumer())
    
    await prod
    await queue.join()
    cons.cancel()
    
await main()

Положили 0 в очередь
Обрабатываем 0


/var/folders/ht/tyf5jl1s1zq2068m9z0n2fnr0000gn/T/ipykernel_1849/3249345357.py:15: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(2)


Положили 1 в очередь
Обрабатываем 1
Положили 2 в очередь
Обрабатываем 2
Положили 3 в очередь
Обрабатываем 3
Положили 4 в очередь
Обрабатываем 4
